# Core Model Guide: LinearTank

This notebook provides a detailed guide to the `LinearTank` model, which is the most fundamental storage model in the CHS SDK. It's used to represent any storage element where the relationship between water level and storage volume is linear (i.e., it has a constant surface area), such as a vertical-walled tank or a simplified reservoir.

## 1. Theoretical Background

The `LinearTank` model is based on the principle of mass conservation. The change in storage volume over a time step (`dt`) is equal to the total inflow minus the total outflow.

```
d(Volume) / dt = Inflow - Outflow
```

For a tank with a constant surface area (`A`), the volume is simply `Volume = Level * A`. Therefore, the change in level is:

```
d(Level) / dt = (Inflow - Outflow) / A
```

The model uses this simple equation to update its water level at each time step.

## 2. API and Parameters

Let's start by importing the necessary libraries and the model itself.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

# Add the project root to the path to allow imports from chs_sdk
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.modules.modeling.storage_models import LinearTank

The `LinearTank` is initialized with the following parameters:

- `area` (float): The constant surface area of the tank in square meters (m²).
- `initial_level` (float): The water level at the beginning of the simulation in meters (m).
- `min_level` (float, optional): The minimum allowed water level. Defaults to 0.
- `max_level` (float, optional): The maximum allowed water level. Defaults to 999.

## 3. Code Example

Let's create an example where we simulate a tank over 100 time steps. We'll start with a constant inflow and then simulate a sudden increase in inflow to see how the tank level responds.

In [ ]:
# 1. Model Initialization
tank = LinearTank(
    area=1000.0,      # 1000 m^2 area
    initial_level=5.0 # Starting at 5m deep
)

# 2. Simulation Setup
dt = 3600  # Time step of 1 hour (3600 seconds)
simulation_duration = 100 # hours
n_steps = simulation_duration

history = {
    "time": [],
    "inflow": [],
    "level": []
}

# 3. Simulation Loop
for t in range(n_steps):
    # Define a time-varying inflow
    if t < 50:
        inflow = 1.0 # 1 m^3/s for the first 50 hours
    else:
        inflow = 2.0 # Double the inflow for the last 50 hours
    
    # Set the inputs for the current time step
    tank.input.inflow = inflow
    tank.input.release_outflow = 0.5 # Constant outflow
    
    # Run the model for one step
    tank.step(dt=dt)
    
    # Record the results
    history["time"].append(t * dt / 3600) # Record time in hours
    history["inflow"].append(inflow)
    history["level"].append(tank.level)

print("Simulation complete.")

## 4. Visualization

Now we can plot the results stored in our `history` dictionary to visualize the tank's behavior.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))

# Plot Level on the primary y-axis
color = 'tab:blue'
ax1.set_xlabel('Time (hours)')
ax1.set_ylabel('Water Level (m)', color=color)
ax1.plot(history['time'], history['level'], color=color, label='Water Level')
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

# Create a second y-axis to plot the inflow
ax2 = ax1.twinx()
color = 'tab:green'
ax2.set_ylabel('Inflow (m³/s)', color=color)
ax2.plot(history['time'], history['inflow'], color=color, linestyle='--', label='Inflow')
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.title('LinearTank Response to Changing Inflow')
plt.show()

The plot clearly shows the water level rising at a constant rate for the first 50 hours. When the inflow doubles at hour 50, the slope of the water level plot increases, indicating a faster rise, as expected.